# Models
From the EDA, I was able to think of my dataset in terms of the officers, the suspects, and the location in which the stops happened. 

In [1]:
# Set Up
import pandas as pd
import numpy as np

# These lines make warnings look nicer
import warnings
warnings.simplefilter('ignore', FutureWarning)

# Graphing
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (10,10)
import seaborn as sns
import plotly as px


# Loading Data

In [70]:
sqf = pd.read_csv('../data/cleaned_one_hot.csv').drop('Unnamed: 0', axis = 1)

# Changing columns names to something more comprehendible 
sqf = sqf.rename({'Other1':'Other Hair', 'Other2':'Other Hair 2',
                'Black.1': 'Black Eyes', 'Blue':'Blue Eyes',
                'Brown':'Brown Eyes', 'Green':'Green Eyes',
                'Gray': 'Gray Eyes', 'Hazel': 'Hazel Eyes',
                'Multicolor': 'Multicolor Eyes', 'Other4': 'Other Eyes 4',
                'Other5': 'Other Eyes 5', 'Other6':'Other Eyes 6'}, axis = 1)


# Outline
Outcome Variable: Frisking (1/0)

Feature Sets
1. race
2. appearance (race + physical characteristics)
3. full context (aforementioned + environmental influences)

Models
1. Logistic Regression (Baseline)
2. Logistic Regression with Regularization (avoiding overfitting)
3. SVM

Statistics
1. Accuracy
2. Precision
3. Recall

Goal: a 3x9 table that describes all combinations of the models, featuresets, and statstics (see results.xlxs)

Steps
1. Wrangle the data to create my feature sets
2. Describe the assumptions behind each model
3. Run the function (generalizeable thanks to sklearn)
4. Compile

# Summary Tables

In [71]:
race_summary = pd.DataFrame(columns = ['Accuracy', 'Precision', 'Recall'], 
                       index = ['Logistic', 'Logisitic + Regularization', 'SVM'])
appearance_summary = pd.DataFrame(columns = ['Accuracy', 'Precision', 'Recall'], 
                       index = ['Logistic', 'Logisitic + Regularization', 'SVM'])

context_summary = pd.DataFrame(columns = ['Accuracy', 'Precision', 'Recall'], 
                       index = ['Logistic', 'Logisitic + Regularization', 'SVM'])

# Wrangle
This section will create 3 dataframes
* race contains racial identifying features of the suspect
* appearance contains any physical features of the suspect, including race
* context contains all possible descriptors the environment in which the officer stopped the suspect

In [72]:
y_raw = sqf['frisked_flag']

race = pd.concat([sqf.iloc[:, 23:29]], axis = 1)
appearance = pd.concat([sqf.iloc[:,19:58]], axis = 1)

# One-hot encoding the NY bourough for context 
borough = pd.get_dummies(sqf['stop_location_boro_name'])
context = pd.concat([appearance, borough], axis = 1)

To speed up the convergence process while running models, I will now normalize these dataframes

In [73]:
def standardize(df):
    """
    A quick, and simple standardization function that relies just on pandas
    Input: dataframe
    Output: dataframe, standardized
    """
    return (df-df.mean())/df.std()

race = standardize(race)
appearance = standardize(appearance)
context = standardize(appearance)

context.head()

,suspect_reported_age,suspect_height,suspect_weight,female,Indian/Alaskan Native,Asian / Pacific Islander,Black,Black Hispanic,White,White Hispanic,...,Black Eyes,Blue Eyes,Brown Eyes,Green Eyes,Gray Eyes,Hazel Eyes,Multicolor Eyes,Other Eyes 4,Other Eyes 5,Other Eyes 6
0,0.149665,-1.584446,1.959845,-0.316528,-0.022893,-0.151021,0.816695,-0.302741,-0.313514,-0.505984,...,-0.328827,-0.121369,0.445248,-0.082342,-0.044994,-0.08984,-0.012234,-0.019346,-0.012234,-0.193017
1,-0.522424,-1.584446,0.728874,-0.316528,-0.022893,-0.151021,0.816695,-0.302741,-0.313514,-0.505984,...,-0.328827,-0.121369,0.445248,-0.082342,-0.044994,-0.08984,-0.012234,-0.019346,-0.012234,-0.193017
2,2.333951,0.583581,2.575330,-0.316528,-0.022893,-0.151021,-1.224355,-0.302741,3.189407,-0.505984,...,-0.328827,8.238711,-2.245772,-0.082342,-0.044994,-0.08984,-0.012234,-0.019346,-0.012234,-0.193017
3,-0.606435,0.583581,-0.502098,-0.316528,-0.022893,-0.151021,0.816695,-0.302741,-0.313514,-0.505984,...,3.040889,-0.121369,-2.245772,-0.082342,-0.044994,-0.08984,-0.012234,-0.019346,-0.012234,-0.193017
4,-0.942479,0.583581,-0.502098,-0.316528,-0.022893,-0.151021,0.816695,-0.302741,-0.313514,-0.505984,...,3.040889,-0.121369,-2.245772,-0.082342,-0.044994,-0.08984,-0.012234,-0.019346,-0.012234,-0.193017


# Great let's begin modeling 
## Logisitc Regression

In [74]:
# BLM
np.random.seed(7132013)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def logistic_reg(df):
    """
    Create a logistic regression classifier of a dataframe
    Input: the feature matrix, which includes the output variable in the first column
    Output: the accuracy, precision, and recall of the matrix

    """

    
    X = df
    y = y_raw
    
    # 80/20 Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
    
    log_reg = LogisticRegression()
    log_reg.fit(X_train, y_train)
    y_pred = log_reg.predict(X_test)
    
    accuracy = log_reg.score(X_test, y_test)
    precision = classification_report(y_test, y_pred, output_dict = True)['weighted avg']['precision']
    recall = classification_report(y_test, y_pred, output_dict = True)['weighted avg']['recall']
    
    return [accuracy, precision, recall]

race_summary.iloc[0,:] = logistic_reg(race)
appearance_summary.iloc[0,:] = logistic_reg(appearance)
context_summary.iloc[0,:] = logistic_reg(context)

In [77]:
display(race_summary,appearance_summary, context_summary)

,Accuracy,Precision,Recall
Logistic,0.580247,0.565221,0.580247
Logisitic + Regularization,NaN,NaN,NaN
SVM,NaN,NaN,NaN


,Accuracy,Precision,Recall
Logistic,0.594089,0.600498,0.594089
Logisitic + Regularization,NaN,NaN,NaN
SVM,NaN,NaN,NaN


,Accuracy,Precision,Recall
Logistic,0.608305,0.61157,0.608305
Logisitic + Regularization,NaN,NaN,NaN
SVM,NaN,NaN,NaN


# Logistic Regression with Normalization

# SVM

#### Observations

What does precision mean in this context? Keep in mind that we are thinking of our machine as a model for an officer in which they are determining if they should frisk a suspect based off their physical appearance. The precision of 0.61 means that of all frisked individuals, our appearance model was able to correctly predict if someone would be frisked based on how they look about 60% of the time. In terms of real-world behavior, this shows that officers are looking at more than just appearance - perhaps they are using their unconscious bias.

Overall this leads to a greater discussion about the **fairness of policing**. From a data science and decision theory perspective, we can _consider_ fair policing to mean the following:

* each race is equally likely to be frisked (equal positve rates)
* each race is equally likely to not be frisked (equal error rates)
* each race is equally likely to be frisked on reasonable suspicion, but then no other consequences, say arrest, occurs (equal false positive rates)

**However**, policing is not about being fair across races as much as it is meant to keep peace, enforce the laws, and uphold justice. SQF is not fair.

In [83]:
from sklearn.svm import SVC
def support_VM(df):
    """
    Create a  classifier of a dataframe
    Input: the feature matrix, which includes the output variable in the first column
    Output: the accuracy, precision, and recall of the matrix

    """

    
    X = df
    y = y_raw
    
    # 80/20 Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
    

    svc = SVC(kernel='linear')
    svc.fit(X_train, y_train)
    
    accuracy = svc.score(X_test, y_test)
    precision = classification_report(y_test, y_pred, output_dict = True)['weighted avg']['precision']
    recall = classification_report(y_test, y_pred, output_dict = True)['weighted avg']['recall']
    
    return [accuracy, precision, recall]

support_VM(race)

ValueError: Found input variables with inconsistent numbers of samples: [2673, 1337]

# Archives

In [10]:
# from sklearn.metrics import roc_auc_score
# from sklearn.metrics import roc_curve
# log_reg_roc_auc = roc_auc_score(y_test, log_reg.predict(X_test))
# fpr, tpr, thresholds = roc_curve(y_test, log_reg.predict_proba(X_test)[:,1])
# plt.figure()
# plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % log_reg_roc_auc)
# plt.plot([0, 1], [0, 1],'r--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic')
# plt.legend(loc="lower right")
# plt.savefig('Log_ROC')
# plt.show()